### Python多线程

**模拟多任务**

In [12]:
from time import ctime,sleep

def music():
    for i in range(3):
        print("i was listening to music. %s" %ctime())
        sleep(1)
def movie():
    for i in range(3):
        print("i was watching movie. %s" %ctime())
        sleep(2)
if __name__=='__main__':
    music()
    movie()
    print("all over %s" %ctime())

i was listening to music. Wed Apr 19 18:18:03 2017
i was listening to music. Wed Apr 19 18:18:04 2017
i was listening to music. Wed Apr 19 18:18:05 2017
i was watching movie. Wed Apr 19 18:18:06 2017
i was watching movie. Wed Apr 19 18:18:08 2017
i was watching movie. Wed Apr 19 18:18:10 2017
all over Wed Apr 19 18:18:12 2017


**带参数的多任务**

**注意：输出结果的时间间隔分别是1秒和2秒，是顺序执行的**

In [16]:
from time import ctime,sleep

def music(func):
    for i in range(3):
        print("i was listening to %s. %s" %(func,ctime()))
        sleep(1)
def movie(func):
    for i in range(3):
        print("i was watching %s. %s" %(func,ctime()))
        sleep(2)
if __name__=='__main__':
    music(u'因为爱情')
    movie(u'大话西游')
    print("all over %s" %ctime())

i was listening to 因为爱情. Wed Apr 19 18:22:33 2017
i was listening to 因为爱情. Wed Apr 19 18:22:34 2017
i was listening to 因为爱情. Wed Apr 19 18:22:35 2017
i was watching 大话西游. Wed Apr 19 18:22:36 2017
i was watching 大话西游. Wed Apr 19 18:22:38 2017
i was watching 大话西游. Wed Apr 19 18:22:40 2017
all over Wed Apr 19 18:22:42 2017


### 多线程实现
Python由模块支持实现多线程，thread（在python3中改名为_thread）,threading,和 queue模块。<br>
therad相对底层，适合高手，threading适合大部分场景下使用<br>
通过加入queue模块，用户可以创建多个线程共享数据的队列数据结构。

注意子程序调用时间，他们是同时被调用的

Thread.setDaemon()和Thread.join()刚好相反，具体看下一小节<br>
如下进程：<br>
main是父进程<br>
music和movie是同级的的子线程<br>
main-->{music,movie}<br>


In [47]:
import threading
from time import ctime,sleep
def music(func):
    for i in range(4):
        print("I was listening to %s. %s\n" %(func,ctime()))
        sleep(1)

def movie(func):
    for i in range(4):
        print("I was at the %s! %s\n" %(func,ctime()))
        sleep(2)
# 这是一个普通List变量，本身与threading模块没有关系，但是这里是用来存放子线程的
threads=[]
# 开始用threading模块的方法开始建立子线程，一个线程就是即将准备运行的一段程序
# 建立了两个线程
# 注意args是一个元组，且即使只有一个参数，结尾也要加逗号','

t1=threading.Thread(target=music,args=(u'因为爱情',))
t2=threading.Thread(target=movie,args=(u'大话西游',))

# 把线程对象放入列表，一遍同时传递
threads.append(t1)
threads.append(t2)

if __name__ =='__main__':
    for t in threads:
#         把线程设置为守护进程，否则会无限挂起
# 线程的挂起：
# 程将挂起，则该线程处于静止状态，如果程序正在运行，将暂停执行，若处于就绪状态，则该线程暂不接受调度，与挂起对应的时激活状态。
        t.setDaemon(True)
        t.start()
#     这里的巧妙在于，在这里调用t.jion()，这个时候t指的是movie，同时movie又是运行时间较长的，
# 所以看上去父进程是等待music和movie都结束了再开始，其实它只等了movie进程，可以试着把它的时间缩短对比下就明白了
#     t.join()
    print('all over %s\n'%ctime())
        

all over Wed Apr 19 20:32:25 2017
I was listening to 因为爱情. Wed Apr 19 20:32:25 2017
I was at the 大话西游! Wed Apr 19 20:32:25 2017



I was listening to 因为爱情. Wed Apr 19 20:32:26 2017

I was listening to 因为爱情. Wed Apr 19 20:32:27 2017

I was at the 大话西游! Wed Apr 19 20:32:27 2017

I was listening to 因为爱情. Wed Apr 19 20:32:28 2017

I was at the 大话西游! Wed Apr 19 20:32:29 2017

I was at the 大话西游! Wed Apr 19 20:32:31 2017



### Join()和setDaemon()

**join():**<br>
在主线程A中，创建了子线程B；<br>
如果不调用B.join(),则主线程A不会等到B运行结束才开始运行，而是同时开始；<br>
如果调用B.join(),则主线程会在B.join()被调用的位置原地等待，知道子线程B运行完(或者等待直到join([timeout])所指定的超时时间之后)，再继续自己的运行

In [35]:
import threading  
import time  
class MyThread(threading.Thread):  
        def __init__(self,id):  
                threading.Thread.__init__(self)  
                self.id = id  
        def run(self):  
                x = 0  
                print('myThread at: %s'%ctime()) 
                time.sleep(5)  
                print('myThread id: %s\n'%self.id)  
  
if __name__ == "__main__":  
        t1=MyThread(999)  
        t1.start()  
        t1.join()
        print('\nmain procudure starts running: %s'%ctime())
        for i in range(5):  
                print(i)  

myThread at: Wed Apr 19 20:16:12 2017
myThread id: 999

main procudure starts running: Wed Apr 19 20:16:17 2017

0
1
2
3
4


**setDaemon()**：<br>
在主线程A中，创建了子线程B；<br>
并且在主线程A中调用了B.setDaemon(),这个的意思是，把主线程A设置为守护线程（**注意逻辑：调用了B.setDaemon()，反而设置的是A**）；<br>
这时候，要是主线程A执行结束了，就不管子线程B是否完成,一并和主线程A退出.这就是setDaemon方法的含义，这基本和join是相反的。
> 此外，还有个要特别注意的：必须在start() 方法调用之前设置，如果不设置为守护线程，程序会被无限挂起。

In [4]:
import threading  
import time  
class MyThread(threading.Thread):  
        def __init__(self,id):  
                threading.Thread.__init__(self)  
        def run(self):  
                time.sleep(5)  
                print("This is " + self.getName()  )
   
if __name__ == "__main__":  
        t1=MyThread(999)  
        t1.setDaemon(True)  
        t1.daemon=True
        t1.start()  
        print("I am the father thread." ) 

I am the father thread.
This is Thread-9


但是这里子线程在主线程结束后，依旧继续执行，原因是我们现在是**交互式模式**，就是一行一行代码的模式，这个模式下只有退出Python.exe，主线程才算被结束了；
另外一种模式叫**脚本模式**，就是把Python代码保存为.py的代码文件，然后用Python xxx.py,这个时候就能体现效果。

上面的threading模块确实实现了多线程，但是：由于GIL全局锁的作用，在同一时间，Python依旧只能让一个线程运行，性能并不能提升，而且也无法利用多核CPU的性能。<br>
那么，上面的多线程不能全面提升性能，只能在特定场景起作用，比如爬虫爬网页的时候的并行下载计算。

### 注意这一段代码无法正常运行

多线程这条路走不通，那么还有一条路，就是并行计算，就是在电脑上同时启动多个Python.exe进程，这本质上不是多线程，而是多进程，类似于玩游戏时候的多开。<br>
多进程的操作模块是multiprocessing，它可以实现并行计算，但是可能是因为有些bug,我测试的时候无法正常运行，google搜到也有人说这个模块有bug

In [ ]:
import multiprocessing as mp

def foo(q):
    q.put('hello')

if __name__ == '__main__':
    mp.set_start_method('spawn')
    q = mp.Queue()
    p = mp.Process(target=foo, args=(q,))
    p.start()
    print(q.get())
    p.join()

### pp模块实现并行计算

#### 介绍、安装

pp模块，就是Parallel Python。分为Python3.x和Python2.x版本，下载下来，然后安装，为什么不直接使用pip install pp?这个我试了，安装失败，所以直接下载源文件进行安排。然后通过setup.py进行安装，先进入下载的包的目录(setup.py所在目录)，然后执行python setup.py install。<br>
[官网](http://www.parallelpython.com/content/view/18/32/)<br>
#### 相关资源
- http://rfyiamcool.blog.51cto.com/1030776/1357112
- http://blog.csdn.net/rumswell/article/details/7324518
- http://www.bubuko.com/infodetail-899846.html

#### 普通模式

In [1]:
# sum_primes_without_pp.py  
import math, sys, time  
  
def isprime(n):  
    """Returns True if n is prime and False otherwise"""  
    if not isinstance(n, int):  
        raise TypeError("argument passed to is_prime is not of 'int' type")  
    if n < 2:  
        return False  
    if n == 2:  
        return True  
    max = int(math.ceil(math.sqrt(n)))  
    i = 2  
    while i <= max:  
        if n % i == 0:  
            return False  
        i += 1  
    return True  
  
def sum_primes(n):  
    """Calculates sum of all primes below given integer n"""  
    return sum([x for x in range(2,n) if isprime(x)])  
  
  
start_time = time.time()  
inputs = (100000, 100100, 100200, 100300, 100400, 100500, 100600, 100700)  
for input in inputs:  
    print("Sum of primes below", input, "is", sum_primes(input)  )
  
print("Time elapsed: ", time.time() - start_time, "s" ) 

Sum of primes below 100000 is 454396537
Sum of primes below 100100 is 454996777
Sum of primes below 100200 is 455898156
Sum of primes below 100300 is 456700218
Sum of primes below 100400 is 457603451
Sum of primes below 100500 is 458407033
Sum of primes below 100600 is 459412387
Sum of primes below 100700 is 460217613
Time elapsed:  2.680223226547241 s


#### 并行计算模式对比
下面这段程序线用普通模式，依次计算，输出计算时间，然后开启并行计算，再次输出计算时间进行对比
程序运行成功的话，打开任务管理器，会发现进程里多出了很多个python.exe进程，这就是并行计算，而且他们是被分配到每个独立的CPU核心的，如果计算量大，可以同时占用所有CPU。

job_server.submit((目标函数)，(目标函数参数)，(目标函数调用的函数)，(目标函数所调用的模块))<br>
job = job_server.submit(function, (paras,), (called-functions,), (imports,))<br>
参数全部必须是tuple类型

In [2]:
#-*- coding: UTF-8 -*-
import math, sys, time
import pp
def IsPrime(n):
    """返回n是否是素数"""
    if not isinstance(n, int):
        raise TypeError("argument passed to is_prime is not of 'int' type")
    if n < 2:
        return False
    if n == 2:
        return True
    max = int(math.ceil(math.sqrt(n)))
    i = 2
    while i <= max:
        if n % i == 0:
            return False
        i += 1
    return True
def SumPrimes(n):
    for i in range(15):
        sum([x for x in range(2,n) if IsPrime(x)])
    """计算从2-n之间的所有素数之和"""
    return sum([x for x in range(2,n) if IsPrime(x)])
inputs = (100000, 100100, 100200, 100300, 100400, 100500, 100600, 100700)
start_time = time.time()
for input in inputs:
    print(SumPrimes(input))
print('单线程执行，总耗时', time.time() - start_time, 's')
# tuple of all parallel python servers to connect with
ppservers = ()
#ppservers = ("10.0.0.1",)
if len(sys.argv) > 1:
#     ncpus = int(sys.argv[1])
#     指定CPU核心数，如果程序计算量很大，建议不要全部用满，否则可能出现电脑卡顿的现象，可以留出一个核心给系统
#     比如你是8核CPU，最大使用7个就好
    ncpus=8
    # Creates jobserver with ncpus workers
#     开启服务
    job_server = pp.Server(ncpus, ppservers=ppservers)
else:
    # Creates jobserver with automatically detected number of workers
#     自动识别系统CPU核心数目，自动生成对应数量的进程
    job_server = pp.Server(ppservers=ppservers)
print("pp 可以用的工作核心线程数", job_server.get_ncpus(), "workers")
start_time = time.time()
# job_server.submit()函数，它的每一个参数都必须是元组，即tuple，
# 所以要加上括号()，而且，即使只有一个值，后面也要跟上逗号，如果没有值，放一个空的括号即可
# 每个参数的含义，去官网查看即可
jobs = [(input, job_server.submit(SumPrimes,(input,), (IsPrime,), ("math",))) for input in inputs]
# 进程生成了之后，还没有开始执行，所以需要调用启动它，调用的方法如下，当做函数来调用，即把进程对象名字加上一个括号即可。
for input, job in jobs:
    print("Sum of primes below", input, "is", job())
print("多线程下执行耗时: ", time.time() - start_time, "s")
# 输出并行计算性能数据
job_server.print_stats()

454396537
454996777
455898156
456700218
457603451
458407033
459412387
460217613
单线程执行，总耗时 41.97292137145996 s
pp 可以用的工作核心线程数 8 workers
Sum of primes below 100000 is 454396537
Sum of primes below 100100 is 454996777
Sum of primes below 100200 is 455898156
Sum of primes below 100300 is 456700218
Sum of primes below 100400 is 457603451
Sum of primes below 100500 is 458407033
Sum of primes below 100600 is 459412387
Sum of primes below 100700 is 460217613
多线程下执行耗时:  11.16439151763916 s
Job execution statistics:
 job count | % of all jobs | job time sum | time per job | job server
         8 |        100.00 |      88.7608 |    11.095101 | local
Time elapsed since server creation 11.164905786514282
0 active tasks, 8 cores



#### 其他注意事项
- 以上是一个Demo，实际应用的话，要考虑自己程序的实际，合理设置参数
- 特别是依赖的包，尽量使用原名，因为开启多进程后，每个进程就独立运行了，无法像一个Python一样顺利找到这些包，比如，我们通常用import pandas as pd，然后使用pd.read_csv()这样的方式，通常这样的写法pp模块会提示无法找到'pd'模块，因为开启多个进程后，这个别名无法正常找到，所以，直接改成pandas.read_csv()就可以了。
- 另外是变量，也要注意，如果出错，提升某个变量未定义，直接尝试不用变量，或者在程序内部就近定义变量<br>
> 下面这个是我自己的提取用户行为数据的例子，大家和官网文档一起参照体会，很快就能懂了

In [ ]:
import pp
from time import ctime
import time
import pandas

auu=pd.read_csv(datas[9])
datas=['actions_all_u.csv','actions_sort_by_user_id.csv','JData_Action_201602.csv','JData_Action_201603.csv','JData_Action_201604.csv',
       'JData_Comment.csv','JData_Product.csv','JData_User.csv','users1.csv','ac_after_4_10.csv','user_after_4_10.csv']
data4=['user_after_4_10_2.csv','user_after_4_10_3.csv','user_after_4_10_4.csv','user_after_4_10_5.csv','user_after_4_10_6.csv']
def extract_user(f):
    auu=pandas.read_csv('ac_after_4_10.csv')
    for i in f.index:
        x=f.iloc[i].user_id
        print(x)
        auu.loc[auu['user_id']==x].to_csv(r".\u_after_410\U_%d.csv"%x)

# 进程元组
ppservers = ()
#   number of kernels of this computer
ncpus=5
# Creates jobserver with ncpus workers
job_server = pp.Server(ncpus, ppservers=ppservers)
# Creates jobserver with automatically detected number of workers
#job_server = pp.Server(ppservers=ppservers)
print("pp 可以用的工作核心线程数", job_server.get_ncpus(), "workers")
start_time = time.time()
jobs=[]
for u in data4:
    f=pandas.read_csv(u)
    jobs.append(job_server.submit((extract_user),(f,),(),("pandas",)))
    
for job in jobs:
    job()
print("多线程下执行耗时: ", time.time() - start_time, "s")
job_server.print_stats()
print('done %s'%ctime())